In [ ]:
import pandas as pd
import string

ttl_data = pd.read_json('tracktranslegislation.json')
aclu_data = pd.read_json('aclu.json')
aclu_data['state'] = aclu_data['state'].transform(lambda x: x['value'])

In [ ]:
aclu_bills = aclu_data.copy()
#aclu_bills['state'] = aclu_bills['state'].transform(lambda x: x['value'])
aclu_bills['name'] = aclu_bills['name'].transform(lambda x: x.replace(' ', '').replace('23-', ''))
aclu_bills = aclu_bills[['state', 'name']]

ttl_bills = ttl_data[['state', 'billId']].rename(columns={'billId': 'name'})
ttl_bills

merged = aclu_bills.merge(ttl_bills, indicator=True, how='outer')
# merged[merged['_merge'] == 'left_only']
pd.set_option('display.max_rows', None)
merged[merged['_merge'] != 'both'].sort_values(by=['state', 'name'])

aclu_only = merged[merged['_merge'] == 'left_only']
def match_suffix(num: str, billId: str):
    if not billId.endswith(num):
        return False
    
    search = billId.rstrip(num)[::-1]
    for item in search:
        if item in '123456789':
            return False
    
    return True

def advanced_check(row):
    state = row['state']
    ttl_for_state = list(ttl_bills.loc[ttl_bills.state == state]['name'])
    prefix, num = row['name'].split(' ')
    prefix = prefix.strip(string.punctuation)
    candidates = sorted([
        bill 
        for bill 
        in ttl_for_state 
        if (
            bill[0] == prefix[0]
            and match_suffix(num, bill)
        )
    ])
    
    return len(candidates) == 0

#for idx, row in aclu_data.loc[aclu_only.index].iterrows():
#    if advanced_check(row):
#        pass
#        print('\t'.join([state, prefix, num, '|',  *candidates]))
        

# match_suffix('1674', 'S1674')
first_filter = aclu_data.loc[aclu_only.index]
second_filter = first_filter[first_filter.apply(lambda x: advanced_check(x), axis=1)]
second_filter.sort_values(['state', 'name'])

print(len(aclu_data))
print(len(ttl_data))
print(len(second_filter))
second_filter